In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [ ]:
SERVER0 = ''
SERVER1 = ''
SERVER2 = ''

In [5]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 5336)
address_rst = (SERVER2, 3344)

ppl = PipelineCommon([
    (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [6]:
text = 'При этом отправители письма напрямую связывают удары американских беспилотников с '\
'серией терактов в Париже, которые произошли 13 ноября 2015 года. '\
'«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, '\
'зная, какие разрушительные последствия за рубежом имеет программа использования '\
'БПЛА», - говорится в открытом письме экс-военных.'


In [8]:
%%time

result = ppl(text)

CPU times: user 22 ms, sys: 0 ns, total: 22 ms
Wall time: 3.17 s


In [9]:
result['rst']

In [10]:
print(result['rst'][0])

id: 10
text: При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года. «Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.
proba: 0.15137706510373972
relation: elaboration_r
nuclearity: _
left: При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года.
right: «Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.
start: 0
end: 359


In [11]:
def extr_pairs(tree):
    pp = []
    
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

In [12]:
extr_pairs(result['rst'][0])

[['При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже, которые произошли 13 ноября 2015 года.',
  '«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА», - говорится в открытом письме экс-военных.',
  'elaboration_r'],
 ['При этом отправители письма напрямую связывают удары американских беспилотников с серией терактов в Париже,',
  'которые произошли 13 ноября 2015 года.',
  'elaboration_r'],
 ['«Мы не можем спокойно сидеть и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА»,',
  '- говорится в открытом письме экс-военных.',
  'attribution_r'],
 ['«Мы не можем спокойно сидеть',
  'и наблюдать за такими трагедиями, как атаки в Париже, зная, какие разрушительные последствия за рубежом имеет программа использования БПЛА»,',
  'joint_m